In [1]:
import damask
import numpy as np 
import pandas as pd

In [2]:
%config Completer.use_jedi = False

# 1. create input files (geometery, material, load)

## geometry file

In [39]:
# geometry file (3d periodic grid)
size = np.ones(3)*1e-5
cells = [8,8,8]
N_grains = 4
seeds = damask.seeds.from_random(size,N_grains,cells)
grid = damask.GeomGrid.from_Voronoi_tessellation(cells,size,seeds)
grid.save(f'Polycystal_{N_grains}_{cells[0]}x{cells[1]}x{cells[2]}')
grid

cells:  8 × 8 × 8
size:   1e-05 × 1e-05 × 1e-05 m³
origin: 0.0   0.0   0.0 m
# materials: 4

## material file

In [40]:
# material file has three sections (__homogenization__, __phase__, __material__)
mat = damask.YAML(homogenization={}, phase={}, material={})

In [41]:
hom_single = {
    'SX': {                            # (1) name of homogenization
        'N_constituents': 1,          # (2) components number
        'mechanical': {'type': 'pass'} # (3) type of homogenization
    }
}

mat.update(homogenization=hom_single) # add to material.yaml file

In [42]:
phase_alu = {
    'Aluminium':{                                    # (1) name of your phase
        'lattice': 'cF',                             # (2) lattice specific
        'mechanical': {                              # (3) mechanical properties
            'output': ['F', 'P', 'F_e', 'L_p', 'O'], # Your desired output quantities
            'elastic': {},
            'plastic': {}
        }
    }
}

In [43]:
alu_elast = damask.YAML.load('../DAMASK_dev/examples/config/phase/mechanical/elastic/Hooke_Al.yaml')
alu_plast = damask.YAML.load('../DAMASK_dev/examples/config/phase/mechanical/plastic/phenopowerlaw_Al.yaml')

In [44]:
phase_alu['Aluminium']['mechanical']['elastic']['type'] = alu_elast['type']
phase_alu['Aluminium']['mechanical']['elastic']['C_11'] = alu_elast['C_11']
phase_alu['Aluminium']['mechanical']['elastic']['C_12'] = alu_elast['C_12']
phase_alu['Aluminium']['mechanical']['elastic']['C_44'] = alu_elast['C_44']
phase_alu['Aluminium']['mechanical']['plastic'] = dict(alu_plast)

mat.update(phase=phase_alu) # add to material.yaml file

In [47]:
# material section -> grain orientation, components and etc
## generate a simple random texture and add it to material section
rnd = damask.Rotation.from_random(N_grains)

In [48]:
mat_config = damask.ConfigMaterial(**mat)
mat_config.pop('material') # material_add() requires no 'material' key exsists

{}

In [49]:
# each material is single crystal (homogenization) and alu (phase)
mat_config = mat_config.material_add(O=rnd,phase='Aluminium',homogenization='SX') 
mat_config.save('material.yaml')

## load file

In [50]:
## handy snippet to account for exclusiveness of stress/strain BC
def make_P(F,fill=0):
    return [make_P(i,fill) if isinstance(i,list) else\
            fill if i == 'x' else 'x' for i in F]

In [51]:
# each load file contains (1) solver option, (2) several loadsteps 
load_config = damask.LoadcaseGrid(
    solver={'mechanical':'spectral_basic'}, # (1) solver
    loadstep=[]                             # (2) loadsteps                     
)

In [52]:
# each load step specifies (1) BC (stress or strain), (2) time discretization, (3) output frequency
## load step 1 (tensile in x) ##
dot_F = [[1.e-3, 0 , 0 ],
         [   0 ,'x', 0 ],
         [   0 , 0 ,'x']]

loadstep = {
    'boundary_conditions':{              # (1) BC
        'mechanical':{                
            'dot_F':dot_F,
            'P':make_P(dot_F)
        }
    },
    'discretization':{'t':10.,'N':40},   # (2) time discretization
    'f_out':4                            # (3) output frequency
}

load_config['loadstep'].append(loadstep)

## load step 2 (tensile in x) ##
dot_F = [[1.e-3, 0 , 0 ],
         [   0 ,'x', 0 ],
         [   0 , 0 ,'x']]

loadstep = {
    'boundary_conditions':{              # (1) BC
        'mechanical':{                
            'dot_F':dot_F,
            'P':make_P(dot_F)
        }
    },
    'discretization':{'t':60.,'N':60},   # (2) time discretization
    'f_out':4                            # (3) output frequency
}

load_config['loadstep'].append(loadstep)
load_config.save('load.yaml')

right now you have a similar input file named as 
- `Polycystal_{N_grains}_{cells}.vti`
- `material.yaml`
- `load.yaml`

in your folder now, which resemble the simple example in `DAMASK/examples/grid`


# 2. run simulation with DAMASK-grid solver

In [54]:
fn_vti = f'Polycystal_{N_grains}_{cells[0]}x{cells[1]}x{cells[2]}.vti'
fn_load = 'load.yaml'
fn_material = 'material.yaml'

In [57]:
%timeit
# DAMASK_grid needs to be in BIN_PATH
run_log = damask.util.run(f'DAMASK_grid -g {fn_vti} -l {fn_load}  -m {fn_material}')

running 'DAMASK_grid -g Polycystal_4_8x8x8.vti -l load.yaml  -m material.yaml' in './'


In [66]:
print(''.join(run_log))


 <<<+-  parallelization init  -+>>>

 Open MPI v4.1.2, package: Debian OpenMPI, ident: 4.1.2, repo rev: v4.1.2, Nov 24, 2021
 MPI standard: 3.1
 OpenMP version: 201511

 MPI processes: 1
 OMP_NUM_THREADS: 4

 <<<+-  CLI init  -+>>>

     _/_/_/      _/_/    _/      _/    _/_/      _/_/_/  _/    _/    _/_/_/
    _/    _/  _/    _/  _/_/  _/_/  _/    _/  _/        _/  _/            _/
   _/    _/  _/_/_/_/  _/  _/  _/  _/_/_/_/    _/_/    _/_/          _/_/
  _/    _/  _/    _/  _/      _/  _/    _/        _/  _/  _/            _/
 _/_/_/    _/    _/  _/      _/  _/    _/  _/_/_/    _/    _/    _/_/_/
 Grid solver

 F. Roters et al., Computational Materials Science 158:420–478, 2019
 https://doi.org/10.1016/j.commatsci.2018.04.030


 Version: 3.0.0-beta-1-g300a57d5f

 Compiled with: GCC version 11.4.0
 Compiled on: Linux-5.15.90.1-microsoft-standard-WSL2
 Compiler options: -cpp -I /home/yi/app/petsc-3.16.4/include -I /home/yi/app/petsc-3.16.4/linux-gnu/include -I /usr/lib/x86_64-linux-g

# 3. process result and visualize

In [ ]:
result_file = 'add_field_data/20grains16x16x16_tensionX.hdf5'
result = damask.Result(result_file)
result.add_stress_Cauchy()
result.add_strain()

# list data (undeformed configuration)
result.add_equivalent_Mises('sigma')
result.add_equivalent_Mises('epsilon_V^0.0(F)')

# list data (undeformed configuration)
result.view(increments=0)

In [ ]:
result.export_simulation_setup()
grid_file = '20grains16x16x16.vti'
grid   = damask.Grid.load(grid_file)
# loop over all increments and store P(1,1) per grain and avg(F(1,1))
data = {g:pd.DataFrame() for g in grains}
for inc in result.get(['F','P']).values():
    P = inc['P']
    F = inc['F']
    for g in grains:
        points = grid.material.flatten(order='F')==g
        P_11 = P[points,0,0].flatten()
        F_11 = np.broadcast_to(np.average(F[:,0,0]),P_11.shape)
        x = pd.DataFrame({'F_11':F_11,'P_11':P_11})
        data[g] = pd.concat((data[g],x),ignore_index=True)

In [ ]:
for g in grains:
   plot = sns.lineplot(y='P_11',x='F_11',data=data[g])

fig = plot.get_figure()

In [ ]:
# export and visualize with paraview
result_select = result.view_in_times()
result_select.export()